In [1]:
import numpy as np
from keras.models import load_model, Model

Using TensorFlow backend.


## Features from pretrained feature extractors

In [2]:
def get_indices(net_name, mode):
    filenames = np.load('../input/features/' + net_name + '/filenames_' + mode +'.npy')
    indices = np.argwhere(np.array(list(map(lambda fn: fn.split('/')[1][0] != '_', filenames))) == True)
    indices = indices.reshape((1, indices.shape[0]))[0]
    return indices

def get_features_from_nets(net_names, mode, steps):
    indices = get_indices('res_net', mode)
    classes = np.load('../input/features/res_net/classes_' + mode + '_make-model.npy')[indices]
    classnames = np.load('../input/features/res_net/classnames_' + mode + '_make-model.npy')[indices]
    filenames = np.load('../input/features/res_net/filenames_' + mode + '.npy')[indices]
    features = []
    for net_name in net_names:
        net_features = np.load('../input/features/' + net_name + '/features_' + mode +'.npy')[indices]
        features.append(net_features)
    features = np.concatenate(features, axis=1)
    features = np.expand_dims(features, axis=1)
    features = np.repeat(features, steps, axis=1)
    return features, classes, classnames, filenames

In [3]:
features_train, classes_train, classnames_train, filenames_train = get_features_from_nets(['res_net', 'inception', 'xception'], 'train', 3)
features_dev, classes_dev, classnames_dev, filenames_dev = get_features_from_nets(['res_net', 'inception', 'xception'], 'dev', 3)

## Features from LSTM

In [4]:
model = load_model('../input/model/car-identification.h5')
model = Model(inputs=model.input, outputs=model.layers[6].output)

/opt/conda/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
predictions_train = model.predict(features_train, verbose=1)
predictions_dev = model.predict(features_dev, verbose=1)

996/996 [==============================] - 5s 5ms/step


In [6]:
np.save('../input/features/siamese/features_train.npy', predictions_train)
np.save('../input/features/siamese/classes_train.npy', classes_train)
np.save('../input/features/siamese/classnames_train.npy', classnames_train)
np.save('../input/features/siamese/filenames_train.npy', filenames_train)
np.save('../input/features/siamese/features_dev.npy', predictions_dev)
np.save('../input/features/siamese/classes_dev.npy', classes_dev)
np.save('../input/features/siamese/classnames_dev.npy', classnames_dev)
np.save('../input/features/siamese/filenames_dev.npy', filenames_dev)